In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

**Data Cleaning**

In [ ]:
df_cat = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 
                   'ext_col', 'int_col', 'accident', 'clean_title']
    

In [ ]:
def clean_data(df, threshold=100):
    
    cat_columns = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 
                   'ext_col', 'int_col', 'accident', 'clean_title']
    
    for col in cat_columns:
        value_counts = df[col].value_counts(dropna=False)
        low_freq_mask = df[col].map(value_counts) < threshold
        df.loc[low_freq_mask, col] = 'noise'
        
        df[col] = df[col].fillna('missing').astype('category')
    
    return df

df_tr = clean_data(df_train)
df_te = clean_data(df_test)

In [ ]:
df_tr.isnull().sum()

In [ ]:
df_tr.drop('id', axis = 1 , inplace = True)

# Model Buidling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = df_tr.drop('price', axis = 1)
y = df_tr['price']

In [ ]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le = LabelEncoder()
scaler =StandardScaler()

In [ ]:
for col in df_cat:
    x[col] = le.fit_transform(x[col])

In [ ]:
x[x.columns] = scaler.fit_transform(x[x.columns])

**Label Encoded and Standardized Data**

In [ ]:
x.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(random_state=42)

In [ ]:
rfr.fit(x_train,y_train)

In [ ]:
y_pred = rfr.predict(x_test)

**Accuracy Evaluation**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

Test data Mean Square Error

In [ ]:
mse = mean_squared_error(y_test,y_pred)
mse

In [ ]:
RMSE = mse**0.5

In [ ]:
print('Root Mean Squared Error fot test set is :', RMSE)

# Predicting on test dataset provided

In [ ]:
df_te.head()

In [ ]:
df_te.isnull().sum()

as its already cleaned, No Null Value present

In [ ]:
df_te.drop('id', axis = 1 , inplace = True)

In [ ]:
df_te.head(3)

In [ ]:
for col in df_cat:
    df_te[col] = le.fit_transform(df_te[col])

In [ ]:
df_te[df_te.columns] = scaler.fit_transform(df_te[df_te.columns])

In [ ]:
df_te.head()

In [ ]:
pred = rfr.predict(df_te)

In [ ]:
Sub = pd.DataFrame({
    'id': submission.id,
    'price': pred
})

Sub.to_csv('Submission_SD_1.csv', index=False)